In [ ]:
'''
yes24 review crawler
20 reviews for each book
for book that has reviews more than 20
'''

In [1]:
import os
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd

In [2]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.chrome.options import Options
from selenium.webdriver import ActionChains

In [3]:
def crawlReview(house, isbnNum):
    values = list()
    try:
        #isbn
        isbnBlock = driver.find_element_by_css_selector('#infoset_specific > div.infoSetCont_wrap > div > table > tbody > tr:nth-child(3) > td')
        if isbnBlock:
            print('isbn found')
            isbnNum = isbnBlock.text
            print(isbnNum)
            
        reviewPageNum = 4
        for i in range(4):
            print('review {} page'.format(i+1))
            fiveReview(house, isbnNum)
            nextReview = driver.find_element_by_css_selector(f'#infoset_reviewContentList > div.review_sort.sortBot > div.review_sortLft > div > a:nth-child({reviewPageNum})')
            if nextReview:
                nextReviewClick = ActionChains(driver).move_to_element(nextReview).click()
                nextReviewClick.perform()
                time.sleep(10)
                print('successfully clicked next page')
                reviewPageNum = reviewPageNum + 1
            
    except Exception as e:
        print(e)

In [4]:
def fiveReview(house, isbnNum):
    reviewBlock = driver.find_element_by_id('infoset_reviewContentList')
    if reviewBlock:
        print('reviewBlock found')

    childNum = 3
    while(True):
        if childNum==8:
            break

        eachReview = reviewBlock.find_element_by_css_selector(f'div:nth-child({childNum}) > div.reviewInfoBot.crop > a')
        print('each review found')

        #더보기 누르고
        btnClick = ActionChains(driver).move_to_element(eachReview).click()
        btnClick.perform()
        print('successfully clicked btn')
        time.sleep(5)

        #내용가져오고

        reviewTxt = reviewBlock.find_element_by_css_selector(f'div:nth-child({childNum}) > div.reviewInfoBot.origin > div.review_cont')
        reviewContent = reviewTxt.text
        #print(reviewContent)
        house.append([isbnNum, reviewContent])

        #스크롤 조금 내리고
        driver.execute_script("window.scrollTo(0, window.scrollY + 400);")

        childNum = childNum + 1

In [5]:
def forOnePage(house, isbnNum):
    for i in range(10):
        oddNum = 2*i + 1
        whichBook = driver.find_element_by_css_selector(f'#category_layout > tbody > tr:nth-child({oddNum}) > td.goodsTxtInfo > p.review > a')
        if whichBook:
            print(f'{i} book found')

            #check how many reviews
            reviewCnt = int(whichBook.text[:-1])
            print('review : ', reviewCnt)

            if reviewCnt < 20:
                print('less than 20')
                continue

            actions = ActionChains(driver).move_to_element(whichBook).click()
            actions.perform()
            crawlReview(house, isbnNum)

        driver.back()

In [6]:
options = Options()
#options.add_argument("headless")
options.add_argument('start-maximized')
options.add_argument('disable-infobars')
options.add_experimental_option("useAutomationExtension", False)
options.add_experimental_option("excludeSwitches",["enable-automation"])
driver = webdriver.Chrome(options=options, executable_path='chrome driver 경로를 입력해주세요')

try:
    values = list()
    isbn = 0
    pageNum = 1
    
    while(True):
        if pageNum==3:
            break
        
        print('pageNum : ', pageNum)
        #open yes24 page
        driver.get(f'http://www.yes24.com/24/category/bestseller?CategoryNumber=001001009003&sumgb=06&PageNumber={pageNum}')
        forOnePage(values, isbn)
        pageNum = pageNum + 1
        
except Exception as e:
    print(e)

pageNum :  1
0 book found
review :  4
less than 20
1 book found
review :  13
less than 20
2 book found
review :  60
isbn found
9791190073073
review 1 page
reviewBlock found
each review found
successfully clicked btn
each review found
successfully clicked btn
each review found
successfully clicked btn
each review found
successfully clicked btn
each review found
successfully clicked btn
successfully clicked next page
review 2 page
reviewBlock found
each review found
successfully clicked btn
each review found
successfully clicked btn
each review found
successfully clicked btn
each review found
successfully clicked btn
each review found
successfully clicked btn
successfully clicked next page
review 3 page
reviewBlock found
each review found
successfully clicked btn
each review found
successfully clicked btn
each review found
successfully clicked btn
each review found
successfully clicked btn
each review found
successfully clicked btn
successfully clicked next page
review 4 page
reviewBlock

In [7]:
#store values in dataframe
df = pd.DataFrame(values, columns=['ISBN13', 'review'])
print(df.head(5))

          ISBN13                                             review
0  9791190073073    내용이 알차고 풍부하고 다양한 관점이 반영된 책이다. 강추하는 서적입니다. 건강...
1  9791190073073  코로나로 여행도 잘 가지 못하는 상황에서 위안삼아 구입한 책입니다. 연말에 코로나 ...
2  9791190073073  책을배송받아서 보았는데요 결론적으로 10분도읽을가치가없네요내용도 다른 책에있는거 대...
3  9791190073073     아무래도 시국이 시국이다 보니 해외여행은 꿈도 못꾸고 그저 국내 여행이라도 가...
4  9791190073073  이번에 작성하게 될 서평은 "에이든 국내여행 가이드북" 이라는 책이다.\n코로나로 ...


In [8]:
#dataframe to csv
df.to_csv('yes24Review_KoreanTravelP1_2.csv', encoding='utf-8-sig', index=False)

In [120]:
'''
def reviewPos():
    #-----go to review position-----
    #scroll down 1000
    driver.execute_script("window.scrollTo(0, 1000)")
    #find the review tab
    reviewTab = driver.find_element_by_css_selector('#yDetailTabNavWrap > div > div.gd_tabBar > ul > li:nth-child(2) > a')
    if reviewTab:
        print('reveiwTab found')
        #click
        reviewTabClick = ActionChains(driver).move_to_element(reviewTab).click()
        reviewTabClick.perform()

        driver.execute_script("window.scrollTo(0, window.scrollY + 500);")
    #------------------------  
'''

'\ndef reviewPos():\n    #-----go to review position-----\n    #scroll down 1000\n    driver.execute_script("window.scrollTo(0, 1000)")\n    #find the review tab\n    reviewTab = driver.find_element_by_css_selector(\'#yDetailTabNavWrap > div > div.gd_tabBar > ul > li:nth-child(2) > a\')\n    if reviewTab:\n        print(\'reveiwTab found\')\n        #click\n        reviewTabClick = ActionChains(driver).move_to_element(reviewTab).click()\n        reviewTabClick.perform()\n\n        driver.execute_script("window.scrollTo(0, window.scrollY + 500);")\n    #------------------------  \n'